In [2]:
import graphlab as gl
gl.canvas.set_target('ipynb')
import matplotlib.pyplot as plt
from PodcastFeatureEngineering import Podcast_Features
%matplotlib inline
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import imp
import os
from collections import Counter

2016-04-20 21:34:10,368 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1461188049.log


This non-commercial license of GraphLab Create is assigned to crystal.humphries@systemsbiology.org and will expire on November 04, 2016. For commercial licensing options, visit https://dato.com/buy/.


In [3]:
loc = os.path.join('/home/ubuntu/PodcastRecommender/', "src/twitter_followers_list.py")
tweets = imp.load_source('get_twitter_followers', loc)
tweets = tweets.get_twitter_followers()
tweets.get_overlapping_users()
item_sim_mat = tweets.Item_sim_matrix(max_num=20000)

In [4]:
sf = gl.SFrame(item_sim_mat)

In [5]:
(train_set, test_set) = sf.random_split(0.8, seed=1)

In [6]:
items = gl.load_sframe("../data/items_podcast_sframe.gl/")

In [7]:
train_set.head(2)

Title,user_id
For The Show MIA,user_2488
For The Show MIA,user_13980


In [6]:
PF = Podcast_Features()
df = PF.cleaned_df

/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3066: DtypeWarning: Columns (14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
clean_podcast_csv.py:24: FutureWarning: the coerce=True keyword is deprecated, use errors='coerce' instead
  self.df['DateFirstEpisode'] = self.df.DateFirstEpisode.apply(lambda x: pd.to_datetime(str(x), coerce=True))


Creating Tf-idf


clean_podcast_csv.py:25: FutureWarning: the coerce=True keyword is deprecated, use errors='coerce' instead
  self.df['DateLastEpisode']  = self.df.DateLastEpisode.apply(lambda x: pd.to_datetime(str(x), coerce=True))


In [8]:
df.HostingDomain.describe()

count     155804
unique     24471
top       <null>
freq       71669
Name: HostingDomain, dtype: object

In [14]:
all_categories = df.Category.apply(lambda x: x.split('/'))

In [22]:
all_categories = df.Category.apply(lambda x: x.split('/'))
temp = [item for sublist in all_categories for item in sublist]
categories = {x:0 for x in temp}

In [27]:
a = np.array(categorys.keys())

In [31]:
len(df.Category)

155804

In [38]:
cats = pd.DataFrame(0, index=np.arange(len([0,4,5])), columns=['random', 'vale'])

In [55]:
cats[['random','vale']].iloc[1] = [1]*2
cats

,random,vale
0,0,0
1,0,1
2,0,0


In [58]:
cats[['random','vale']].loc[2] = [2,2]

In [66]:
cats.loc[0][['random', 'vale']] = 1

In [67]:
cats

,random,vale
0,1,1
1,0,1
2,2,2


In [68]:
all_categories = df.Category.apply(lambda x: x.split('/'))
temp = [item for sublist in all_categories for item in sublist]
categories = {x:0 for x in temp}

cats = pd.DataFrame(0, index=np.arange(len(df.Category.values)), columns=categories.keys())
for i,v in enumerate(df.Category.values):
    ls = v.split('/')
    cats.loc[i][ls] = 1
cats.head()

,Performing Arts,Philosophy,Tech News,Fitness & Nutrition,Podcasting,Games & Hobbies,Science & Medicine,Educational Technology,Arts,Sports & Recreation,...,Gadgets,Literary Criticism,Design,Teaching & Learning,Hobbies,Music,Management,Education,Technology,Islam
0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
all_categories[0]

['Performing Arts', 'Podcasts', 'Arts', 'Comedy']

In [10]:
df.HostingDomain.value_counts().head()

<null>                     71669
traffic.libsyn.com         10161
feeds.soundcloud.com        6628
media.blubrry.com           5030
recordings.talkshoe.com     1395
Name: HostingDomain, dtype: int64

In [13]:
dates=[]
for a,b in  zip(PF.df.DateLastEpisode, PF.df.DateFirstEpisode):
    try:
        new = (a-b).days
    except:
        new = 'NaT'
    dates.append(new)
dates = np.array(dates)

In [14]:
g = df.columns.to_series().groupby(df.dtypes).groups
c = g.keys()
df2 = df.drop(g[c[0]], axis=1)
df2 = df2.drop(g[c[1]], axis=1)

In [15]:
df2['Title'] = df.Title
df2['dates'] = dates

In [26]:
def get_top_100(col, df):
    tfidf = TfidfVectorizer( max_features=100, stop_words='english')
    X = tfidf.fit_transform(df[col])
    return pd.DataFrame(X.toarray(),
                        columns=pd.Series(tfidf.get_feature_names()))

In [19]:
unique_pod = item_sim_mat.Title.unique()
df2.head()

,TotalEpisodes,RatingCount,AverageRating,CategoryRanking,Title,dates
0,5,4227,4.5,2,The Moth Podcast,28
1,50,2195,4.5,1,Fresh Air,57
2,210,5520,5.0,3,99% Invisible,1951
3,8,851,4.5,4,Selected Shorts,63
4,26,1797,4.5,8,The Black Tapes,242


In [21]:
df_reduced = df2[ df2.Title.isin(unique_pod)]
df_reduced.shape
items_reduced = gl.SFrame(df_reduced)

In [22]:
items_reduced.head()

TotalEpisodes,RatingCount,AverageRating,CategoryRanking,Title,dates
8,851,4.5,4,Selected Shorts,63
26,1797,4.5,8,The Black Tapes,242
52,298,4.5,7,The Splendid Table,350
11,941,4.5,9,Limetown,143
10,407,4.5,12,The Writer's Almanac withGarrison Keillor ...,9
404,129,4.5,13,New Yorker: Out Loud,3033
298,135,4.5,16,KEXP Presents Music ThatMatters ...,2093
100,445,4.5,18,The Dinner Party Download,622
10,678,5.0,19,TANIS,112
288,556,4.5,22,Philosophy Bites,3162


In [24]:
item_model = gl.item_similarity_recommender.create(train_set, user_id="user_id", 
                                                   item_id="Title", item_data=items_reduced, verbose=False)
item_model.evaluate_precision_recall(test_set, cutoffs=[1,2,3,4,5], verbose=False)

Recsys training: model = item_similarity

{'precision_recall_by_user': Columns:
 	user_id	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 84170
 
 Data:
 +---------+--------+----------------+--------+-------+
 | user_id | cutoff |   precision    | recall | count |
 +---------+--------+----------------+--------+-------+
 |  user_0 |   1    |      1.0       |  0.5   |   2   |
 |  user_0 |   2    |      1.0       |  1.0   |   2   |
 |  user_0 |   3    | 0.666666666667 |  1.0   |   2   |
 |  user_0 |   4    |      0.5       |  1.0   |   2   |
 |  user_0 |   5    |      0.4       |  1.0   |   2   |
 |  user_1 |   1    |      0.0       |  0.0   |   1   |
 |  user_1 |   2    |      0.0       |  0.0   |   1   |
 |  user_1 |   3    |      0.0       |  0.0   |   1   |
 |  user_1 |   4    |      0.0       |  0.0   |   1   |
 |  user_1 |   5    |      0.0       |  0.0   |   1   |
 +---------+--------+----------------+--------+-------+
 [84170 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can 

In [27]:
iTunes_Summary = get_top_100("iTunesSummary", df)

In [34]:
df3 = items_reduced.to_dataframe()

In [35]:
df3['dates'] = df3['dates'].apply(lambda x: -1 if x=='NaT' else x)
df3['dates'] = df3['dates'].astype('int')

In [37]:
for col in ['iTunesSummary', 'iTunesKeywords', 'Category']:
    y = get_top_100(col, df)
    df3 = pd.concat([df3, y], axis=1)

In [ ]:
#np.duplicates(df3.columns)
from collections import Counter

In [38]:
ls = Counter(df3.columns)

In [39]:
df_final = pd.DataFrame()
for a,b in ls.items():
    if b==1:
        df_final = pd.concat([df_final, df3[a]],axis=1)
    else:
        df_final[a] = df3[[a]].sum(axis=1)

In [45]:
new_cols_items_reduced = df_final[ df_final.Title.isin(unique_pod)]
new_cols_items_reduced.shape
new_cols_items_reduced = gl.SFrame(new_cols_items_reduced)

In [53]:
item_model = gl.item_similarity_recommender.create(train_set, user_id="user_id", 
                                                   item_id="Title", item_data=new_cols_items_reduced, verbose=False)
results = item_model.evaluate_precision_recall(test_set, cutoffs=[1,2,3,4,5], verbose=False)
new_res = results['precision_recall_overall']

Recsys training: model = item_similarity

In [52]:
item_model = gl.item_similarity_recommender.create(train_set, user_id="user_id", 
                                                   item_id="Title", verbose=False)
results = item_model.evaluate_precision_recall(test_set, cutoffs=[1,2,3,4,5], verbose=False)
orig_res = results['precision_recall_overall']

Recsys training: model = item_similarity

In [74]:
temp_items =new_cols_items_reduced.copy()
temp_items = temp_items.remove_column("Title")

In [ ]:
precision = []
prec_num  = []
recall    = []
recall_num= []

for col in new_cols_items_reduced.column_names()[:-1]:
    cols = ['Title']
    if col == 'Title':
        continue
    cols.append(col)
    item_model = gl.item_similarity_recommender.create(train_set, user_id="user_id", 
                                                       item_id="Title", item_data=new_cols_items_reduced[cols],
                                                       verbose=False)
    new_res = item_model.evaluate_precision_recall(test_set, cutoffs=[1,2,3,4,5], verbose=False)
    new_res = new_res['precision_recall_overall']
    compare_p = sum(orig_res['precision']<new_res['precision'])
    if compare_p>0:
        precision.append(col)
        prec_num.append(compare_p)
        print "Precision" + str(col)
    compare_r = sum(orig_res['recall']<new_res['recall'])
    if compare_r>0:
        recall.append(col)
        prec_num.append(compare_r)
        print "Recall" + str(col)

Recsys training: model = item_similarity

Recsys training: model = item_similarity

Recalltrance
Recalloutdoor

Recsys training: model = item_similarity


Recallhelp

Recsys training: model = item_similarity

Recsys training: model = item_similarity

Recsys training: model = item_similarity

Recsys training: model = item_similarity

Recsys training: model = item_similarity

In [75]:
precision

[]

In [65]:
results['precision_recall_overall']['recall']

dtype: float
Rows: 5
[0.17010089980326695, 0.2576731340837931, 0.31014825614183394, 0.34243752479832257, 0.3629584509068477]

In [ ]:
kept_columns = []
def compare(orig_res, new_res, col):
    comparison = orig_res['precision']<new_res['precision']
    if sum(comparison)>0:
        kept_columns.append(col)
        

In [ ]:
df_final = pd.read_pickle("item_all.pkl")
df_num = df_final.drop(["Title", "summaries"], axis=1)

In [ ]:
items = pd.read_json('random')

In [ ]:
users = gl.SFrame.read_csv('random1_t')

In [ ]:
sf1 = sf[sf['summaries']==sum_ar[0]]

In [ ]:
names = pd.read_pickle("users_10orMorePodcst.pkl")
names = gl.SArray(names.index.tolist())
#users.add_column(names, 'Title')

In [ ]:
users = gl.load_sframe('df_user_info.gl')

In [ ]:
users.show()

In [ ]:
users.head()

In [ ]:
users.shape

In [ ]:
main_df = sf1[['TotalEpisodes','Title','summaries']]

In [ ]:
item_desccription = sf1.remove_column('TotalEpisodes')

In [ ]:
a = users['user_id']

In [ ]:
ls = ['user_id_']*len(a)

In [ ]:
users['user_id'] = gl.SFrame([ z+str(int(b)) for z,b in zip(ls,a)])

In [ ]:
t = gl.SArray([ z+str(int(b)) for z,b in zip(ls,a)])

In [ ]:
users['user_id'] = t

In [ ]:
a = gl.item_similarity_recommender.create(users, user_id="user_id", item_id="Title", 
                                           item_data=items)

In [ ]:
a.get_similar_items()

In [ ]:
##need to do this!
percentage_english = df.iTunesSummary.apply(lambda x: pods._find_percentage(x))
df2 = df[percentage_english>=0.6]

In [ ]:
summaries = PF.get_new_column_of_text('iTunesKeywords')

In [ ]:
items = gl.SFrame(df_final)

In [ ]:
df_final['summaries']=pd.Series([' '.join(x) for x in summaries[0]])
#sum_ar

In [ ]:
gl.item_similarity_recommender??